## Features:

- brand
- model
- speed
- brake type 
- time til end
- current_bid_cnt
- frame size / stand over height
- frame material
- rim material
- current price

In [196]:
import pandas as pd
import glob
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [34]:
def append_data(csv):
    '''
    combines individual csv files
    '''
    df_list = []
    
    for i in csv:
        
        df = pd.read_csv(i)
        df_list.append(df)
        
    combined = pd.concat(df_list)
    
    return combined


def main():
    
    return df 

In [254]:
files = glob.glob('*.csv')
df = append_data(files)
print('Shape of df: ', df.shape)

Shape of df:  (1943, 44)


In [255]:
df['brand'] = [x.split(' ')[0].lower() if len(x) > 0 else x for x in df['name'].tolist()]
df['model'] = [x.split(' ')[1].lower() if len(x) > 0 else x for x in df['name'].tolist()]

# df['brand'].unique()
# manually created
brand_list = ['trek', 'giant', 'norco', 'northrock', 'columbia', 'ccm',
       'nishiki', 'movelo', 'schwinn', 
       'specialized', 'timberline',
       'apollo', 'marin', 'nakamura', 'swift', 'supercycle',
       'devinci', 'khs', 'diamondback', 'debranded', 'hyper',
       'diadora', 'reebok',
       'bianchi', 'huffy', 'arashi', 'mongoose',
        'raleigh', 'cannondale', 'gt', 'kona', 'renegade']

df = df[df['brand'].isin(brand_list)]
df = df[~df['model'].str.contains('bmx')]
df = df[['name', 'brand', 'model', 'timetilend', 'current_bid_cnt', 'frame size', 'stand over height', 'speed', 'gear shifter',
       'front derailleur', 'rear derailleur', 'brake lever', 'brake', 'brake type', 'crank set', 'price']]

# clean columns
df['model'] = df['model'].apply(lambda x: 'no model provided' if 'speed' in x else x)
df['timetilend'] = df['timetilend'].str.strip().str.replace('\n', '').str.replace(',', ' ')
df['gear shifter brand'] = [str(x).split(' ')[0].lower() if len(str(x)) > 0 else x for x in df['gear shifter'].tolist()]
df['gear shifter brand'] = df['gear shifter brand'].str.replace('nan', 'generic').str.replace(';', '')
df['front derailleur brand'] = [str(x).split(' ')[0].lower() if len(str(x)) > 0 else x for x in df['front derailleur'].tolist()]
df['front derailleur brand'] = df['front derailleur brand'].str.replace('nan', 'generic').str.replace(';', '')
df['rear derailleur brand'] = [str(x).split(' ')[0].lower() if len(str(x)) > 0 else x for x in df['rear derailleur'].tolist()]
df['rear derailleur brand'] = df['rear derailleur brand'].str.replace('nan', 'generic').str.replace(';', '')
df['brake lever brand'] = [str(x).split(' ')[0].lower() if len(str(x)) > 0 else x for x in df['brake lever'].tolist()]
df['brake lever brand'] = df['brake lever brand'].str.replace('nan', 'generic').str.replace(';', '')
df['crank set brand'] = [str(x).split(' ')[0].lower() if len(str(x)) > 0 else x for x in df['crank set'].tolist()]
df['crank set brand'] = df['crank set brand'].str.replace('nan', 'geneic').str.replace(';', '').str.replace('sr', 'sr suntour')
df['brake'] = df['brake'].replace(np.nan, 'nan')
df['brake type'] = df['brake type'].replace(np.nan, 'nan')
df['brake type'] = df['brake'] + df['brake type']
df['brake type'] = df['brake type'].apply(lambda x: 'disc' if 'disc' in x else 'rim')
df['speed'] = df['speed'].replace('single', 1).replace('single/fixed gear', 1).replace('fixed gear', 1).replace(np.nan, 0)
df['timetilend'] = pd.to_timedelta(df['timetilend']).dt.total_seconds()/3600/24

In [256]:
df_final = df[['brand', 'model', 'speed', 'timetilend', 'current_bid_cnt', 'stand over height', 'gear shifter brand',
              'front derailleur brand', 'rear derailleur brand', 'brake lever brand', 'crank set brand', 
               'brake type', 'price']]

In [257]:
df_final['gear shifter brand'] = df_final['gear shifter brand'].apply(lambda x: x if ('shimano' in x or 'sram' in x or 'suntour' in x) else 'generic')
df_final['front derailleur brand'] = df_final['front derailleur brand'].apply(lambda x: x if ('shimano' in x or 'sram' in x or 'suntour' in x) else 'generic')
df_final['rear derailleur brand'] = df_final['rear derailleur brand'].apply(lambda x: x if ('shimano' in x or 'sram' in x or 'suntour' in x) else 'generic')
df_final['crank set brand'] = df_final['crank set brand'].apply(lambda x: x if ('shimano' in x or 'sram' in x or 'fsa' in x) else 'generic')
df_final['stand over height'] = df_final['stand over height'].apply(lambda x: str(x)[:4]).str.replace('"', '').str.replace(";", '').astype(float)

In [262]:
df_final['speed'] = df_final['speed'].astype(float)
df_final['stand over height'].fillna((df_final['stand over height'].mean()), inplace=True)

In [259]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1311 entries, 0 to 47
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   brand                   1311 non-null   object 
 1   model                   1311 non-null   object 
 2   speed                   1311 non-null   float64
 3   timetilend              1311 non-null   float64
 4   current_bid_cnt         1311 non-null   int64  
 5   stand over height       1301 non-null   float64
 6   gear shifter brand      1311 non-null   object 
 7   front derailleur brand  1311 non-null   object 
 8   rear derailleur brand   1311 non-null   object 
 9   brake lever brand       1311 non-null   object 
 10  crank set brand         1311 non-null   object 
 11  brake type              1311 non-null   object 
 12  price                   1311 non-null   float64
dtypes: float64(4), int64(1), object(8)
memory usage: 143.4+ KB


In [260]:
df_final.head()

,brand,model,speed,timetilend,current_bid_cnt,stand over height,gear shifter brand,front derailleur brand,rear derailleur brand,brake lever brand,crank set brand,brake type,price
0,trek,crossrip,24.0,0.124769,83,31.0,shimano,shimano,shimano,integrated,fsa,disc,1111.75
1,trek,6000,30.0,0.135150,48,31.0,shimano,shimano,shimano,shimano,shimano,disc,526.75
2,giant,revel,24.0,0.145544,78,31.0,shimano,shimano,shimano,shimano,shimano,disc,432.75
3,norco,storm,24.0,0.155926,136,28.0,shimano,shimano,shimano,tektro,generic,disc,556.75
4,northrock,xc27,21.0,0.166319,118,30.0,generic,shimano,shimano,shimano,shimano,disc,346.75


## Create a regression model

In [309]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from xgboost import XGBRegressor
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor


from lazypredict.Supervised import LazyRegressor
from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np

In [268]:
X = df_final.drop(columns = 'price')
y = df_final['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Output of lazy predict

In [308]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric= None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

print(models)

 37%|███▋      | 16/43 [00:10<00:36,  1.36s/it]

HuberRegressor model failed to execute
'str' object has no attribute 'decode'


 72%|███████▏  | 31/43 [00:13<00:03,  3.52it/s]

PoissonRegressor model failed to execute
'str' object has no attribute 'decode'


 88%|████████▊ | 38/43 [00:15<00:01,  3.12it/s]

StackingRegressor model failed to execute
__init__() missing 1 required positional argument: 'estimators'


100%|██████████| 43/43 [00:16<00:00,  2.65it/s]

                                                                       R-Squared  \
Model                                                                              
ExtraTreesRegressor                                                         0.99   
ExtraTreeRegressor                                                          0.98   
XGBRegressor                                                                0.98   
DecisionTreeRegressor                                                       0.98   
RandomForestRegressor                                                       0.98   
BaggingRegressor                                                            0.97   
LGBMRegressor                                                               0.97   
GaussianProcessRegressor                                                    0.97   
HistGradientBoostingRegressor                                               0.97   
GradientBoostingRegressor                                                   

## Using some of the regressors above

In [318]:
estimators = [('ridge', RidgeCV()), 
              ('svr', LinearSVR()),
              ('lr', LinearRegression()),
              ('dt', DecisionTreeRegressor(random_state=42)), 
              ('rfc', RandomForestRegressor(random_state= 42)),
              ('knn', KNeighborsRegressor()),
              ('XGB', XGBRegressor(random_state=42))
             ]


models = [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor(), KNeighborsRegressor(),
          XGBRegressor(), StackingRegressor(estimators=estimators, final_estimator=XGBRegressor(n_estimators=10, random_state = 42))]

## Create a function to evaluate the model using RMSE

In [316]:
def predict_price(models):
    
    for model in models:
        
        num_proc = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())

        cat_proc = make_pipeline(
            SimpleImputer(strategy='constant', fill_value='missing'),
            OneHotEncoder(handle_unknown='ignore'))

        preprocessor = make_column_transformer((num_proc, ('speed', 'timetilend', 'current_bid_cnt', 'stand over height')),
                                               (cat_proc, ('brand', 'model', 'gear shifter brand', 'front derailleur brand', 
                                                           'rear derailleur brand', 'brake lever brand', 'crank set brand', 
                                                           'brake type')))

        clf = make_pipeline(preprocessor, model)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

    
        print('Model: ', model, 'RMSE: ', mean_squared_error(y_test, y_pred, squared= False), 
              'r2: ', r2_score(y_test, y_pred))

In [319]:
predict_price(models)

Model:  LinearRegression() RMSE:  43.30977621855274 r2:  0.9463039662278199
Model:  DecisionTreeRegressor() RMSE:  32.214710389544614 r2:  0.970291632249801
Model:  RandomForestRegressor() RMSE:  29.3983666424231 r2:  0.9752590294508876
Model:  KNeighborsRegressor() RMSE:  44.91551983084607 r2:  0.9422485105822513
Model:  XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) RMSE:  28.522165379501132 r2:  0.9767118325100512
Model:  StackingRegressor(estimators=[('ridge',
    

In [351]:
df_final.columns

Index(['brand', 'model', 'speed', 'timetilend', 'current_bid_cnt',
       'stand over height', 'gear shifter brand', 'front derailleur brand',
       'rear derailleur brand', 'brake lever brand', 'crank set brand',
       'brake type', 'price'],
      dtype='object')

In [352]:
df_final.columns = ['brand', 'model', 'speed', 'timetilend', 'current bid count',
       'stand over height', 'gear shifter brand', 'front derailleur brand',
       'rear derailleur brand', 'brake lever brand', 'crank set brand',
       'brake type', 'price']

In [353]:
df_final.head()

,brand,model,speed,timetilend,current bid count,stand over height,gear shifter brand,front derailleur brand,rear derailleur brand,brake lever brand,crank set brand,brake type,price
0,trek,crossrip,24.00,0.12,83,31.00,shimano,shimano,shimano,integrated,fsa,disc,1111.75
1,trek,6000,30.00,0.14,48,31.00,shimano,shimano,shimano,shimano,shimano,disc,526.75
2,giant,revel,24.00,0.15,78,31.00,shimano,shimano,shimano,shimano,shimano,disc,432.75
3,norco,storm,24.00,0.16,136,28.00,shimano,shimano,shimano,tektro,generic,disc,556.75
4,northrock,xc27,21.00,0.17,118,30.00,generic,shimano,shimano,shimano,shimano,disc,346.75


# Save best model as pkl

In [322]:
import joblib

In [373]:
num_proc = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())

cat_proc = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore'))

preprocessor = make_column_transformer((num_proc, ('speed', 'timetilend', 'current bid count', 'stand over height')),
                                       (cat_proc, ('brand', 'model', 'gear shifter brand', 'front derailleur brand', 
                                                   'rear derailleur brand', 'brake lever brand', 'crank set brand', 
                                                   'brake type')))

clf = make_pipeline(preprocessor, StackingRegressor(estimators=estimators, 
                                                    final_estimator=XGBRegressor(n_estimators=10,
                                                                                 random_state = 42)))

In [377]:
xgb = make_pipeline(preprocessor, XGBRegressor(n_estimators=10, random_state = 42))

In [356]:
X = df_final.drop(columns = 'price')
y = df_final['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [360]:
X_test.columns

Index(['brand', 'model', 'speed', 'timetilend', 'current bid count',
       'stand over height', 'gear shifter brand', 'front derailleur brand',
       'rear derailleur brand', 'brake lever brand', 'crank set brand',
       'brake type'],
      dtype='object')

In [359]:
clf.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ('speed', 'timetilend',
                                                   'current bid count',
                                                   'stand over height')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 stra

In [378]:
xgb.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ('speed', 'timetilend',
                                                   'current bid count',
                                                   'stand over height')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 stra

In [361]:
joblib.dump(clf, 'model.pkl')

['model.pkl']

In [384]:
joblib.dump(xgb, 'xgb.pkl')

['xgb.pkl']

In [391]:
test = joblib.load('model.pkl')

In [385]:
test2 = joblib.load('xgb.pkl')

In [375]:
test

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ('speed', 'timetilend',
                                                   'current bid count',
                                                   'stand over height')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 stra

## Test case

In [392]:
test.predict(pd.DataFrame([['trek', 'crossgrip', 24, 0.12, 83, 31, 'shimano', 'shimano', 'shimano', 'shimano', 'fsa', 'disc']],
           columns=['brand', 'model', 'speed', 'timetilend', 'current bid count',
       'stand over height', 'gear shifter brand', 'front derailleur brand',
       'rear derailleur brand', 'brake lever brand', 'crank set brand',
       'brake type']))

array([620.68335], dtype=float32)

In [387]:
test2.predict(pd.DataFrame([['trek', 'crossgrip', 24, 0.12, 83, 31, 'shimano', 'shimano', 'shimano', 'shimano', 'fsa', 'disc']],
           columns=['brand', 'model', 'speed', 'timetilend', 'current bid count',
       'stand over height', 'gear shifter brand', 'front derailleur brand',
       'rear derailleur brand', 'brake lever brand', 'crank set brand',
       'brake type']))

array([574.60065], dtype=float32)